In [ ]:
import requests
import pandas as pd

# Foursquare

Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice. 

In [ ]:
# Get CityBike stations from the API
def get_citybike_stations():
    """
    Fetch station data from the CityBike API.
    Returns:
        pd.DataFrame: DataFrame containing station data with uid, latitude, longitude, and free_bikes.
    """
    url = "http://api.citybik.es/v2/networks/bikesampa"
    params = {
        "fields": "stations.extra.uid,stations.latitude,stations.longitude,stations.free_bikes",
    }
    response = requests.get(url, params=params)
    
    if response.status_code == 200:
        data = response.json()
        stations = data['network']['stations']
        
        for station in stations:
            station['uid'] = station['extra']['uid']
        
        return pd.DataFrame(stations, columns=['uid', 'latitude', 'longitude', 'free_bikes'])
    else:
        print(f"CityBike API Error: {response.status_code}")
        print(f"Response Text: {response.text}")
        return pd.DataFrame()

# Get POIs with /places/search
def get_foursquare_pois(lat, lon, category, radius=200):
    url = "https://api.foursquare.com/v3/places/search"
    headers = {
        "Accept": "application/json",
        "Authorization": "xxx"  
    }
    params = {
        "ll": f"{lat},{lon}",
        "radius": radius,
        "categories": category,
        "limit": 10
    }
    response = requests.get(url, headers=headers, params=params, timeout=10)
    
    if response.status_code == 200:
        return response.json().get('results', [])
    else:
        print(f"Foursquare API Error: {response.status_code}")
        print(f"Response Text: {response.text}")
        return []

# Get detailed information for each POI using /details
def get_foursquare_details(fsq_id):
    url = f"https://api.foursquare.com/v3/places/{fsq_id}"
    headers = {
        "Accept": "application/json",
        "Authorization": "xxx"  
    }
    response = requests.get(url, headers=headers, timeout=10)
    
    if response.status_code == 200:
        return response.json()  
    else:
        print(f"Details request failed for POI {fsq_id}: {response.status_code}")
        print(f"Response Text: {response.text}")
        return {}



Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [ ]:
def get_combined_pois(lat, lon, category):
    pois = get_foursquare_pois(lat, lon, category)
    combined_pois = []
    
    for poi in pois:
        fsq_id = poi.get('fsq_id')
        details = get_foursquare_details(fsq_id)  
        print(poi)  
            
        combined_pois.append({
            'fsq_id': fsq_id,
            'name': poi.get('name'),
            'distance': poi.get('distance'),
            'rating': details.get('rating'),
            'popularity': details.get('popularity'),
            'price': details.get('price'),
            'hours': details.get('hours', {}).get('display') if details.get('hours') else None,
            'hours_popular': details.get('hours_popular')
        })
    
    return combined_pois

# Get data for all CityBike stations
categories_foursquare = {
    'restaurants': '13065',
    'coffee shops': '13032',
    'bakeries': '13002'
}

Put your parsed results into a DataFrame

In [ ]:
citybike_df = get_citybike_stations()

if citybike_df.empty:
    print("No CityBike data available. Exiting.")
else:
    combined_data = []

    for _, station in citybike_df.iterrows():
        lat, lon, uid = station['latitude'], station['longitude'], station['uid']
        
        for category_name, category_id in categories_foursquare.items():
            pois = get_combined_pois(lat, lon, category_id)
            for poi in pois:
                combined_data.append({
                    'station_id': uid,
                    'fsq_id': poi.get('fsq_id'),
                    'name': poi.get('name'),
                    'distance': poi.get('distance'),
                    'rating': poi.get('rating'),
                    'popularity': poi.get('popularity'),
                    'price': poi.get('price'),
                    'hours': poi.get('hours'),
                    'hours_popular': poi.get('hours_popular'),
                    'category': category_name
                })

    combined_df = pd.DataFrame(combined_data)

    print(citybike_df)

    print(combined_df)


# Yelp

Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice. 

In [ ]:
# Get CityBike stations from the API
def get_citybike_stations():
    url = "http://api.citybik.es/v2/networks/bikesampa"
    params = {
        "fields": "stations.extra.uid,stations.latitude,stations.longitude,stations.free_bikes",
    }
    response = requests.get(url, params=params)
    
    if response.status_code == 200:
        data = response.json()
        stations = data['network']['stations']
        
        for station in stations:
            if 'extra' in station and 'uid' in station['extra']:
                station['uid'] = station['extra']['uid']
        
        return pd.DataFrame(stations, columns=['uid', 'latitude', 'longitude', 'free_bikes'])
    else:
        print(f"CityBike API Error: {response.status_code}")
        print(f"Response Text: {response.text}")
        return pd.DataFrame()

# Get Yelp POIs
def get_yelp_pois(lat, lon, term, radius=200):
    url = "https://api.yelp.com/v3/businesses/search"
    headers = {
        "Authorization": "Bearer xxx"  
    }
    params = {
        "latitude": lat,
        "longitude": lon,
        "radius": radius,
        "categories": term,
        "limit": 10
    }
    print(f"Making request to Yelp API with parameters: {params}")
    response = requests.get(url, headers=headers, params=params)
    
    if response.status_code == 200:
        return response.json().get('businesses', [])
    else:
        print(f"Yelp API Error: {response.status_code}")
        print(f"Response Text: {response.text}")
        return []

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [ ]:
# Define Yelp categories
categories_yelp = ['restaurants', 'coffee', 'bakeries']

# Get CityBike station data
df_citybike = get_citybike_stations()

if df_citybike.empty:
    print("No CityBike data available. Exiting.")
else:
# Get Yelp POIs for each CityBike station
    yelp_data = []

    for _, station in df_citybike.iterrows():
        lat, lon, uid = station['latitude'], station['longitude'], station['uid']
        
        for term in categories_yelp:
            pois = get_yelp_pois(lat, lon, term)
            for poi in pois:
                yelp_data.append({
                    'station_id': uid,
                    'poi_id': poi.get('id'),
                    'name': poi.get('name'),
                    'category': term,
                    'distance': poi.get('distance'),
                    'rating': poi.get('rating'),
                    'review_count': poi.get('review_count'),
                    'price': poi.get('price', None),
                    'is_closed': poi.get('is_closed', None),
                })


Put your parsed results into a DataFrame

In [ ]:

yelp_df = pd.DataFrame(yelp_data)


print(df_citybike)
print(yelp_df)



# Comparing Results

Which API provided you with more complete data? Provide an explanation. 

The Yelp API had a limit constraint because I was using a free account, so I lost some data but also obtained the fields that I needed. Foursquare returned all the important fields as null.



Get the top 10 restaurants according to their rating

In [ ]:

top_10 = yelp_df.sort_values(by='rating', ascending=False).head(10)

print(top_10['name'])

737          Amor Aos Pedaços
380             Lima Restobar
1673            Saint Germain
1258    Pedacinho do Nordeste
1265              Hot-Dog Usp
405         La Arena Parrilla
1289      Supra di Mauro Maia
1296                    Jaber
382          Riso Restaurante
1302                 Moa Café
Name: name, dtype: object